In [1]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [2]:
import pandas as pd
df = pd.read_csv('preprocessed_lyrics.csv')

df = df.groupby('Genre', group_keys=False).apply(lambda s: s.sample(333, random_state=42)) #




In [3]:
df

,SName,Lyric,Genre,Artist,lyric_length_processed
2157,Number One,uh uh uh I got to bring dirty thas you well wa...,Hip Hop,Nelly,299
3187,Put That Thang On U (ft. Snoop Dogg),Fala lady like lady Ne Yo sing to real quick I...,Hip Hop,Ne-yo,233
9164,The Resurrection of Scott Mescudi,Instrumental once realize free you fly,Hip Hop,Kid Cudi,6
2145,Feel Like Home,these day dark night cold People act like lose...,Hip Hop,Fort Minor,239
6442,Public Service Announcement 2000,this another public bring in by Slim Shady Emi...,Hip Hop,Eminem,51
...,...,...,...,...,...
31824,Stone In My Shoe,Rudolf Schenker lyric Klaus Meine I dancin try...,Rock,Scorpions,101
40499,TIN SOLDIER,I little that want jump you oh look a pass sky...,Rock,Uriah Heep,67
32051,I Am a Stone,it hard that I on straight you be it lung and ...,Rock,Demon Hunter,117
34364,Too Much To Ask,it I feel lonely I wish cure it funny think go...,Rock,Avril Lavigne,130


In [4]:
df["label"] = LabelEncoder().fit_transform(df["Genre"])


In [5]:
import tensorflow as tf
import pandas as pd
import keras

from sklearn import preprocessing
from keras.utils import np_utils
import numpy as np


y = list(df['label'])
x = list(df['Lyric'])

le = preprocessing.LabelEncoder()
le.fit(y)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.np_utils.to_categorical(enc)

def decode(le, one_hot):
    print(one_hot)
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)


x_enc = x
y_enc = encode(le, y)

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(np.asarray(x_enc), np.asarray(y_enc), test_size=0.2, random_state=42)

In [7]:
embeddings_index = {} # Will be a 100dim vector for each word in glove, found as embeddings_index[word]
f = open('glove.6B.100d.txt',encoding='utf8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

400001it [00:10, 39531.58it/s]

Found 400001 word vectors.


## Defining hyperparameters

In [8]:
VOCABULARY_SIZE = 30000
MAX_LENGTH = 500

In [9]:
tokenizer = Tokenizer(num_words=VOCABULARY_SIZE)
tokenizer.fit_on_texts(list(x_train) + list(x_test))

Turning tokens into lists of sequences

In [10]:
xtrain_sequence = tokenizer.texts_to_sequences(x_train)
xtest_sequence = tokenizer.texts_to_sequences(x_test)

In [11]:
xtrain_padding = sequence.pad_sequences(xtrain_sequence, maxlen=MAX_LENGTH)
xtest_padding = sequence.pad_sequences(xtest_sequence, maxlen=MAX_LENGTH)
print(xtest_padding.shape)
word_index = tokenizer.word_index


(200, 500)


In [12]:
embedding_matrix = np.zeros((len(word_index) + 1, 100))
print(embedding_matrix.shape)
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

  0%|          | 0/8570 [00:00<?, ?it/s]

(8571, 100)


100%|██████████| 8570/8570 [00:00<00:00, 517182.03it/s]


- Embedding_matrix is now just a matrix over the embeddings. i.e as embedding_index but in matrix form with 29000 ish rows
- word_index is a mapping of all of the words appearing in lyrics to an integer

## LSTM model with glove embeddings and two dense layers.

We could change dropout percentage to further avoid/less avoid overfitting

In [13]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                    100,
                    weights=[embedding_matrix],
                    input_length=MAX_LENGTH,
                    trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(100, dropout=0.3, recurrent_dropout=0.3)))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [14]:
#ytrain_encode = np_utils.to_categorical(ytrain)
#ytest_encode = np_utils.to_categorical(ytest)

NameError: name 'ytrain' is not defined

In [16]:
import tensorflow as tf
import keras.backend as K

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(xtrain_padding, 
                    y=y_train, 
                    batch_size=16, 
                    epochs=1, 
                    verbose=1)
    predicts = model.predict(xtest_padding, batch_size=16)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
799/799 [==============================] - 67s 84ms/step - loss: 1.1012 - acc: 0.3442


In [ ]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

from sklearn.metrics import accuracy_score

print("Accuracy of ELMO is:",accuracy_score(y_test,y_preds))

In [ ]:
def graph_plots(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history[‘val_’+string])
    plt.xlabel(“Epochs”)
    plt.ylabel(string)
    plt.legend([string, ‘val_’+string])
    plt.show()

graph_plots(history, “accuracy”)
graph_plots(history, “loss”)